In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import requests
from neo4j import GraphDatabase

# Neo4j connection (no auth)
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = ""

# creating a neo4j driver instance
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Groq + LLaMA 3 API config (set it in your env or paste directly here)
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "Paste_your_key_here")
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"
LLAMA_MODEL = "llama3-70b-8192"

# getting schema from neo4j
def get_graph_schema():
    with driver.session() as session:
        labels = set()
        for record in session.run("MATCH (n) RETURN DISTINCT labels(n) AS labels"):
            for label in record["labels"]:
                labels.add(label)
        rels = [r["relationshipType"] for r in session.run("CALL db.relationshipTypes()")]
    return {"labels": list(labels), "relationship_types": rels}

# Run Cypher query
def run_cypher_query(query):
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]

# Generate Cypher with context
def generate_cypher_query(messages):
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": LLAMA_MODEL,
        "messages": messages,
        "temperature": 0,
        "max_tokens": 300
    }
    response = requests.post(GROQ_API_URL, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"].strip()

# Main chat loop
def chat_bot():
    print("🔗 Connected to Neo4j.")
    print("🤖 Chatbot powered by LLaMA 3 (Groq API). Type 'exit' to quit.\n")
    schema = get_graph_schema()

    # Initial context messages
    messages = [
        {"role": "system", "content": f"""You are an assistant that converts natural language questions into Cypher queries for a Neo4j knowledge graph.

Schema details:
Labels: {schema['labels']}
Relationship Types: {schema['relationship_types']}

Only return Cypher queries. Do not explain."""}
    ]

    while True:
        user_input = input("\n🧠 Ask your question (or type 'exit'): ").strip()
        if user_input.lower() == "exit":
            print("👋 Goodbye!")
            break

        messages.append({"role": "user", "content": user_input})
        print("💬 Generating Cypher query...")
        try:
            cypher_query = generate_cypher_query(messages)
            print("\n📄 Cypher Query:\n", cypher_query)
        except Exception as e:
            print("❌ Error generating query:", e)
            continue

        messages.append({"role": "assistant", "content": cypher_query})

        run_it = input("▶️  Run this query? (y/n): ").strip().lower()
        if run_it == "y":
            try:
                results = run_cypher_query(cypher_query)
                print("\n📊 Query Results:")
                for r in results:
                    print(r)
            except Exception as e:
                print("❌ Error running query:", e)
        else:
            print("⏩ Skipped execution.")

# Run the chatbot
if __name__ == "__main__":
    chat_bot()


🔗 Connected to Neo4j.
🤖 Chatbot powered by LLaMA 3 (Groq API). Type 'exit' to quit.

💬 Generating Cypher query...

📄 Cypher Query:
 MATCH (se:SubmodelElement {idShort: 'TypeOfEmailAddress'}) RETURN se;

📊 Query Results:
{'se': {'smId': 'https://smartfactory.de/submodels/1b8b8375-73e6-4f37-9d7c-6d08e21b066f', 'idShort': 'TypeOfEmailAddress', 'idShortPath': 'ContactInformation.Email.TypeOfEmailAddress', 'value': '0173-1#07-AAS754#001'}}
{'se': {'smId': 'https://smartfactory.de/submodels/1b8b8375-73e6-4f37-9d7c-6d08e21b066f', 'idShort': 'TypeOfEmailAddress', 'idShortPath': 'Address.Address.Email01.TypeOfEmailAddress', 'value': 'Office'}}
💬 Generating Cypher query...

📄 Cypher Query:
 MATCH (se:SubmodelElement {idShort: 'TypeOfEmailAddress'}) RETURN se;
⏩ Skipped execution.
💬 Generating Cypher query...

📄 Cypher Query:
 MATCH (se:SubmodelElement {idShort: 'TypeOfEmailAddress'}) RETURN se;
⏩ Skipped execution.
💬 Generating Cypher query...

📄 Cypher Query:
 MATCH (se:SubmodelElement {idShort